In [10]:
import sys
!{sys.executable} -m pip install tenacity



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /opt/miniconda3/bin/python -m pip install --upgrade pip


In [11]:
import json

def load_professor_list(jsonl_path="neu_professor_links.jsonl"):
    """
    Load full professor list from JSONL file and return as a Python list.
    Each line in JSONL is: {"url": "..."}
    """
    professor_list = []

    with open(jsonl_path, "r") as f:
        for line in f:
            try:
                obj = json.loads(line)
                professor_list.append(obj["url"])
            except:
                pass

    return professor_list

# Load and print full list
professor_list = load_professor_list()
print(f"Total URLs: {len(professor_list)}")
print(professor_list)


Total URLs: 3769
['https://www.ratemyprofessors.com/professor/1853272', 'https://www.ratemyprofessors.com/professor/3034822', 'https://www.ratemyprofessors.com/professor/1699685', 'https://www.ratemyprofessors.com/professor/2983522', 'https://www.ratemyprofessors.com/professor/1839408', 'https://www.ratemyprofessors.com/professor/2428661', 'https://www.ratemyprofessors.com/professor/2426138', 'https://www.ratemyprofessors.com/professor/2194624', 'https://www.ratemyprofessors.com/professor/218477', 'https://www.ratemyprofessors.com/professor/2963762', 'https://www.ratemyprofessors.com/professor/123486', 'https://www.ratemyprofessors.com/professor/2745134', 'https://www.ratemyprofessors.com/professor/2854044', 'https://www.ratemyprofessors.com/professor/2998086', 'https://www.ratemyprofessors.com/professor/2935875', 'https://www.ratemyprofessors.com/professor/2729088', 'https://www.ratemyprofessors.com/professor/175273', 'https://www.ratemyprofessors.com/professor/2766310', 'https://www.

In [12]:
"""
RateMyProfessors scraper using BeautifulSoup (+ GraphQL for reviews).

Notes:
- Significantly faster than browser automation (≈50–100×) when critical data
  is present in the initial HTML or via public API endpoints.
- If content is rendered dynamically via JavaScript and no API is exposed,
  identify endpoints first (e.g., via network inspection) before falling back
  to headless browsers.
"""

import json
import asyncio
import logging
import re
import base64
from typing import Optional, Dict, List
import httpx
from bs4 import BeautifulSoup
from tenacity import retry, stop_after_attempt, wait_exponential

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

OUTPUT_FILE = "neu_professors_stream.jsonl"

# GraphQL endpoint used by the RMP frontend (subject to change by the provider).
GRAPHQL_URL = "https://www.ratemyprofessors.com/graphql"

# Conservative rate-limiting defaults to reduce the risk of 429/ban while maintaining throughput.
REQUEST_DELAY = 1.0  # seconds between requests
MAX_RETRIES = 3      # max retries after a 429 response
RATE_LIMIT_BACKOFF = 5  # initial backoff (seconds) with exponential growth

# Rotated desktop UA strings to blend with typical client traffic patterns.
USER_AGENTS = [
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Safari/605.1.15",
]

def get_headers():
    """Return HTTP headers with a randomized User-Agent to mitigate trivial bot heuristics."""
    import random
    return {
        "User-Agent": random.choice(USER_AGENTS),
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Cache-Control": "max-age=0",
        "Referer": "https://www.ratemyprofessors.com/",
    }


async def fetch_page(client: httpx.AsyncClient, url: str, retry_count: int = 0) -> Optional[str]:
    """
    Retrieve raw HTML with backoff-aware handling of transient errors.

    Behavior:
    - Detects 429 responses and retries with exponential backoff (bounded by MAX_RETRIES).
    - Returns the page text on success; None on unrecoverable failures.
    - Uses randomized headers per request to reduce correlation.
    """
    try:
        headers = get_headers()
        response = await client.get(url, headers=headers, timeout=30.0, follow_redirects=True)
        
        # Handle rate limiting explicitly.
        if response.status_code == 429:
            if retry_count < MAX_RETRIES:
                backoff_time = RATE_LIMIT_BACKOFF * (2 ** retry_count)
                logger.warning(
                    f"Rate limited (429) for {url}. "
                    f"Waiting {backoff_time}s before retry {retry_count + 1}/{MAX_RETRIES}..."
                )
                await asyncio.sleep(backoff_time)
                return await fetch_page(client, url, retry_count + 1)
            else:
                logger.error(f"Rate limited (429) for {url} after {MAX_RETRIES} retries. Skipping.")
                return None
        
        # Non-429 client/server errors.
        if response.status_code >= 400:
            logger.error(f"HTTP error {response.status_code} for {url}")
            return None
        
        response.raise_for_status()
        return response.text
        
    except httpx.TimeoutException:
        logger.error(f"Timeout fetching {url}")
        return None
    except httpx.HTTPStatusError as e:
        if e.response.status_code == 429:
            # Defensive guard; primary path above already handles this.
            return None
        logger.error(f"HTTP error {e.response.status_code} for {url}")
        return None
    except Exception as e:
        logger.error(f"Error fetching {url}: {e}")
        return None


def extract_professor_data(soup: BeautifulSoup, url: str) -> Dict:
    """
    Extract instructor metadata from the static DOM.

    Fields populated (best-effort):
    - name, department, overall rating (if present in HTML)
    - reviews (only if review cards are server-rendered; otherwise populated via GraphQL)
    """
    prof = {
        "url": url,
        "title": soup.title.string if soup.title else "",
        "name": "",
        "department": "",
        "rating": "",
        "reviews": []
    }
    
    # Name: fall back across several robust selectors to accommodate class name churn.
    name_selectors = [
        "h1",
        "[class*='Name']",
        "[class*='ProfessorName']",
        ".professor-name"
    ]
    for selector in name_selectors:
        elem = soup.select_one(selector)
        if elem and elem.get_text(strip=True):
            prof["name"] = elem.get_text(strip=True)
            break
    
    # Department: tolerant to minor frontend refactors.
    dept_selectors = [
        "[class*='TeacherDepartment']",
        "[class*='Department']",
        "[class*='dept']"
    ]
    for selector in dept_selectors:
        elem = soup.select_one(selector)
        if elem and elem.get_text(strip=True):
            prof["department"] = elem.get_text(strip=True)
            break
    
    # Overall rating: extract a numeric score if a composite string is encountered.
    rating_selectors = [
        "[class*='RatingValue']",
        "[class*='rating']",
        "[class*='overall-rating']"
    ]
    for selector in rating_selectors:
        elem = soup.select_one(selector)
        if elem and elem.get_text(strip=True):
            rating_text = elem.get_text(strip=True)
            match = re.search(r'(\d+\.?\d*)\s*/?\s*5', rating_text)
            prof["rating"] = match.group(1) if match else rating_text
            break
    
    # Review cards (if server-rendered). Many deployments are client-rendered; expect empty.
    review_selectors = [
        "[class*='RatingCard']",
        "[class*='Rating__']",
        "[class*='review-card']",
        "[class*='ReviewCard']"
    ]
    
    for selector in review_selectors:
        review_cards = soup.select(selector)
        if review_cards:
            logger.info(f"Found {len(review_cards)} reviews using selector: {selector}")
            break
    else:
        review_cards = []
        logger.warning("No review cards found with standard selectors")
    
    for card in review_cards:
        review = {
            "date": "",
            "rating": "",
            "comment": ""
        }
        
        # Comment text.
        comment_selectors = [
            "[class*='Comments']",
            "[class*='comment']",
            "[class*='review-text']"
        ]
        for sel in comment_selectors:
            elem = card.select_one(sel)
            if elem:
                review["comment"] = elem.get_text(strip=True)
                break
        
        # Date (prefer human-readable, fall back to machine attribute if available).
        date_elem = card.select_one("time")
        if date_elem:
            review["date"] = date_elem.get_text(strip=True)
            if not review["date"] and date_elem.get("datetime"):
                review["date"] = date_elem.get("datetime", "")
        
        # Per-review rating (layout varies; use generic class substrings).
        rating_selectors = [
            "[class*='CardNumRating']",
            "[class*='rating']",
            "[class*='score']"
        ]
        for sel in rating_selectors:
            elem = card.select_one(sel)
            if elem:
                review["rating"] = elem.get_text(strip=True)
                break
        
        if review["comment"] or review["date"] or review["rating"]:
            prof["reviews"].append(review)
    
    return prof


def extract_professor_id(url: str) -> Optional[str]:
    """Parse a numeric legacyId from canonical profile URLs (e.g., /professor/2631895)."""
    match = re.search(r'/professor/(\d+)', url)
    return match.group(1) if match else None


def encode_graphql_id(professor_id: str) -> str:
    """
    Convert the numeric legacyId to the Relay-style base64 node ID used by GraphQL.
    Pattern: 'Teacher-{legacyId}' → base64.
    """
    teacher_id = f"Teacher-{professor_id}"
    encoded = base64.b64encode(teacher_id.encode('utf-8')).decode('utf-8')
    return encoded


async def fetch_reviews_via_graphql(
    client: httpx.AsyncClient, 
    graphql_id: str, 
    cursor: Optional[str] = None,
    count: int = 20
) -> Optional[Dict]:
    """
    Invoke the RatingsListQuery to retrieve a page of ratings for a teacher node.

    Returns:
        Parsed JSON dict on success; None on failure or transient 429 (caller handles paging/retry).
    """
    # This query mirrors the frontend’s persisted GraphQL document (subject to change).
    graphql_query = """query RatingsListQuery(
  $count: Int!
  $id: ID!
  $courseFilter: String
  $cursor: String
) {
  node(id: $id) {
    __typename
    ... on Teacher {
      ...RatingsList_teacher_4pguUW
    }
    id
  }
}

fragment CourseMeta_rating on Rating {
  attendanceMandatory
  wouldTakeAgain
  grade
  textbookUse
  isForOnlineClass
  isForCredit
}

fragment NoRatingsArea_teacher on Teacher {
  lastName
  ...RateTeacherLink_teacher
}

fragment ProfessorNoteEditor_rating on Rating {
  id
  legacyId
  class
  teacherNote {
    id
    teacherId
    comment
  }
}

fragment ProfessorNoteEditor_teacher on Teacher {
  id
}

fragment ProfessorNoteFooter_note on TeacherNotes {
  legacyId
  flagStatus
}

fragment ProfessorNoteFooter_teacher on Teacher {
  legacyId
  isProfCurrentUser
}

fragment ProfessorNoteHeader_note on TeacherNotes {
  createdAt
  updatedAt
}

fragment ProfessorNoteHeader_teacher on Teacher {
  lastName
}

fragment ProfessorNoteSection_rating on Rating {
  teacherNote {
    ...ProfessorNote_note
    id
  }
  ...ProfessorNoteEditor_rating
}

fragment ProfessorNoteSection_teacher on Teacher {
  ...ProfessorNote_teacher
  ...ProfessorNoteEditor_teacher
}

fragment ProfessorNote_note on TeacherNotes {
  comment
  ...ProfessorNoteHeader_note
  ...ProfessorNoteFooter_note
}

fragment ProfessorNote_teacher on Teacher {
  ...ProfessorNoteHeader_teacher
  ...ProfessorNoteFooter_teacher
}

fragment RateTeacherLink_teacher on Teacher {
  legacyId
  numRatings
  lockStatus
}

fragment RatingFooter_rating on Rating {
  id
  comment
  adminReviewedAt
  flagStatus
  legacyId
  thumbsUpTotal
  thumbsDownTotal
  thumbs {
    thumbsUp
    thumbsDown
    computerId
    id
  }
  teacherNote {
    id
  }
  ...Thumbs_rating
}

fragment RatingFooter_teacher on Teacher {
  id
  legacyId
  lockStatus
  isProfCurrentUser
  ...Thumbs_teacher
}

fragment RatingHeader_rating on Rating {
  legacyId
  date
  class
  helpfulRating
  clarityRating
  isForOnlineClass
}

fragment RatingSuperHeader_rating on Rating {
  legacyId
}

fragment RatingSuperHeader_teacher on Teacher {
  firstName
  lastName
  legacyId
  school {
    name
    id
  }
}

fragment RatingTags_rating on Rating {
  ratingTags
}

fragment RatingValues_rating on Rating {
  helpfulRating
  clarityRating
  difficultyRating
}

fragment Rating_rating on Rating {
  comment
  flagStatus
  createdByUser
  teacherNote {
    id
  }
  ...RatingHeader_rating
  ...RatingSuperHeader_rating
  ...RatingValues_rating
  ...CourseMeta_rating
  ...RatingTags_rating
  ...RatingFooter_rating
  ...ProfessorNoteSection_rating
}

fragment Rating_teacher on Teacher {
  ...RatingFooter_teacher
  ...RatingSuperHeader_teacher
  ...ProfessorNoteSection_teacher
}

fragment RatingsList_teacher_4pguUW on Teacher {
  id
  legacyId
  lastName
  numRatings
  school {
    id
    legacyId
    name
    city
    state
    avgRating
    numRatings
  }
  ...Rating_teacher
  ...NoRatingsArea_teacher
  ratings(first: $count, after: $cursor, courseFilter: $courseFilter) {
    edges {
      cursor
      node {
        ...Rating_rating
        id
        __typename
      }
    }
    pageInfo {
      hasNextPage
      endCursor
    }
  }
}

fragment Thumbs_rating on Rating {
  id
  comment
  adminReviewedAt
  flagStatus
  legacyId
  thumbsUpTotal
  thumbsDownTotal
  thumbs {
    computerId
    thumbsUp
    thumbsDown
    id
  }
  teacherNote {
    id
  }
}

fragment Thumbs_teacher on Teacher {
  id
  legacyId
  lockStatus
  isProfCurrentUser
}"""
    
    payload = {
        "query": graphql_query,
        "operationName": "RatingsListQuery",
        "variables": {
            "count": count,
            "id": graphql_id,
            "courseFilter": None,
            "cursor": cursor
        }
    }
    
    headers = get_headers()
    headers["Content-Type"] = "application/json"
    headers["Referer"] = "https://www.ratemyprofessors.com/"
    
    try:
        response = await client.post(
            GRAPHQL_URL,
            json=payload,
            headers=headers,
            timeout=30.0
        )
        
        if response.status_code == 429:
            logger.warning(f"Rate limited on GraphQL request. Waiting...")
            await asyncio.sleep(RATE_LIMIT_BACKOFF)
            return None
        
        response.raise_for_status()
        return response.json()
        
    except Exception as e:
        logger.error(f"Error fetching GraphQL data: {e}")
        return None


def parse_graphql_reviews(graphql_response: Dict) -> List[Dict]:
    """
    Normalize a RatingsListQuery page to a list of review dicts.

    Each dict includes:
        - date (ISO or display string)
        - rating (composed from clarity/helpful/difficulty where available)
        - comment (free text)
    """
    reviews = []
    
    try:
        node = graphql_response.get("data", {}).get("node", {})
        if node.get("__typename") != "Teacher":
            return reviews
        
        ratings = node.get("ratings", {})
        edges = ratings.get("edges", [])
        
        for edge in edges:
            review_node = edge.get("node", {})
            
            review = {
                "date": review_node.get("date", ""),
                "rating": "",  # composed below if present
                "comment": review_node.get("comment", "")
            }
            
            # Aggregate component scores (when available) into a single display string.
            rating_parts = []
            if review_node.get("clarityRating"):
                rating_parts.append(f"Clarity: {review_node.get('clarityRating')}")
            if review_node.get("helpfulRating"):
                rating_parts.append(f"Helpful: {review_node.get('helpfulRating')}")
            if review_node.get("difficultyRating"):
                rating_parts.append(f"Difficulty: {review_node.get('difficultyRating')}")
            
            review["rating"] = " | ".join(rating_parts) if rating_parts else ""
            
            if review["comment"] or review["date"]:
                reviews.append(review)
        
        return reviews
        
    except Exception as e:
        logger.error(f"Error parsing GraphQL reviews: {e}")
        return []


async def get_all_reviews_via_graphql(
    client: httpx.AsyncClient,
    professor_id: str,
    max_pages: int = 100
) -> List[Dict]:
    """
    Retrieve all ratings for a given legacyId via GraphQL pagination.

    Parameters:
        client: shared HTTP client
        professor_id: numeric legacyId from the profile URL
        max_pages: hard ceiling on page iteration to avoid runaway loops

    Returns:
        List[Dict] of normalized review entries.
    """
    graphql_id = encode_graphql_id(professor_id)
    all_reviews = []
    cursor = None
    page = 0
    
    while page < max_pages:
        logger.debug(f"Fetching GraphQL page {page + 1} for professor {professor_id}...")
        
        response = await fetch_reviews_via_graphql(client, graphql_id, cursor, count=20)
        
        if not response:
            break
        
        # Parse and append this page’s reviews.
        page_reviews = parse_graphql_reviews(response)
        all_reviews.extend(page_reviews)
        
        # Advance pagination cursor.
        try:
            node = response.get("data", {}).get("node", {})
            ratings = node.get("ratings", {})
            page_info = ratings.get("pageInfo", {})
            has_next = page_info.get("hasNextPage", False)
            cursor = page_info.get("endCursor")
            
            if not has_next or not cursor:
                break
            
            page += 1
            # Short inter-page pause to smooth traffic.
            await asyncio.sleep(0.5)
            
        except Exception as e:
            logger.error(f"Error checking pagination: {e}")
            break
    
    logger.info(f"Fetched {len(all_reviews)} reviews via GraphQL for professor {professor_id}")
    return all_reviews


async def scrape_professor_bs4(client: httpx.AsyncClient, url: str) -> Optional[Dict]:
    """
    End-to-end scraper for a single profile:
      1) Fetch HTML and extract static metadata (name, department, rating).
      2) Retrieve full ratings via GraphQL and attach to the record.

    Returns:
        A dictionary with profile metadata and a list of reviews, or None on failure.
    """
    try:
        logger.info(f"Scraping: {url}")
        
        # Extract legacyId from the canonical URL.
        professor_id = extract_professor_id(url)
        if not professor_id:
            logger.error(f"Could not extract professor ID from {url}")
            return None
        
        # Static HTML for basic fields.
        html = await fetch_page(client, url)
        if not html:
            return None
        
        soup = BeautifulSoup(html, 'html.parser')
        
        # Base metadata.
        prof = extract_professor_data(soup, url)
        
        # Do not rely on server-rendered reviews; fetch via GraphQL instead.
        prof["reviews"] = []
        
        logger.info(f"Fetching all reviews via GraphQL for professor {professor_id}...")
        graphql_reviews = await get_all_reviews_via_graphql(client, professor_id)
        prof["reviews"] = graphql_reviews
        
        logger.info(f"Extracted {len(prof['reviews'])} reviews from {url} (via GraphQL)")
        
        return prof
        
    except Exception as e:
        logger.error(f"Error scraping {url}: {e}", exc_info=True)
        return None


def load_already_scraped() -> set:
    """
    Read previously scraped URLs from OUTPUT_FILE to support resumable runs.

    Returns:
        A set of URLs present in the JSONL output (ignores malformed lines).
    """
    done = set()
    try:
        with open(OUTPUT_FILE, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    obj = json.loads(line)
                    if "url" in obj:
                        done.add(obj["url"])
                except:
                    continue
    except FileNotFoundError:
        pass
    return done


def save_json_stream(record: Dict):
    """
    Append a single JSON record to OUTPUT_FILE as JSONL.

    This streaming pattern avoids holding the entire corpus in memory and
    makes long runs fault-tolerant.
    """
    with open(OUTPUT_FILE, "a", encoding="utf-8") as f:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")


async def scrape_all_bs4(professor_list: List[str], max_concurrent: int = 5):
    """
    Bounded-concurrency scraper for a list of profile URLs.

    Features:
        - Respects a global delay per “slot” to reduce burstiness.
        - Uses a shared AsyncClient with connection pooling (HTTP/2 enabled).
        - Streams results to disk incrementally.

    Args:
        professor_list: iterable of canonical profile URLs
        max_concurrent: semaphore limit for in-flight tasks (default 5)
    """
    already = load_already_scraped()
    logger.info(f"Already scraped: {len(already)}")
    
    remaining = [link for link in professor_list if link not in already]
    logger.info(f"Remaining to scrape: {len(remaining)}")
    
    if not remaining:
        logger.info("All URLs already scraped!")
        return
    
    logger.info(f"Rate limiting: {REQUEST_DELAY}s delay, max {max_concurrent} concurrent requests")
    logger.info(f"Will handle 429 errors with exponential backoff (max {MAX_RETRIES} retries)")
    
    # Pooled client tuned for modest parallelism and reuse.
    async with httpx.AsyncClient(
        timeout=httpx.Timeout(30.0),
        limits=httpx.Limits(
            max_keepalive_connections=max_concurrent,
            max_connections=max_concurrent * 2
        ),
        http2=True,
        follow_redirects=True
    ) as client:
        semaphore = asyncio.Semaphore(max_concurrent)
        saved_count = 0
        failed_count = 0
        rate_limited_count = 0
        total = len(remaining)
        
        # Slot-based pacing array (prevents synchronized bursts).
        last_request_time = [0.0] * max_concurrent
        
        async def scrape_with_limit(url: str, idx: int):
            nonlocal saved_count, failed_count, rate_limited_count
            async with semaphore:
                # Enforce per-slot spacing to maintain REQUEST_DELAY.
                slot_id = idx % max_concurrent
                current_time = asyncio.get_event_loop().time()
                time_since_last = current_time - last_request_time[slot_id]
                if time_since_last < REQUEST_DELAY:
                    await asyncio.sleep(REQUEST_DELAY - time_since_last)
                last_request_time[slot_id] = asyncio.get_event_loop().time()
                
                result = await scrape_professor_bs4(client, url)
                if result:
                    save_json_stream(result)
                    saved_count += 1
                    reviews_count = len(result.get('reviews', []))
                    logger.info(
                        f" [{saved_count}/{total}] Saved: {result.get('name', 'Unknown')} "
                        f"({reviews_count} reviews)"
                    )
                else:
                    failed_count += 1
                    logger.warning(f" [{idx+1}/{total}] Failed: {url}")
                return result
        
        tasks = [scrape_with_limit(url, idx) for idx, url in enumerate(remaining)]
        results = await asyncio.gather(*tasks, return_exceptions=True)
        
        success_count = sum(1 for r in results if r and not isinstance(r, Exception))
        final_failed_count = total - success_count
        
        logger.info("=" * 60)
        logger.info(f" Scraping complete!")
        logger.info(f"   Successfully scraped: {success_count}/{total}")
        if final_failed_count > 0:
            logger.info(f"   Failed: {final_failed_count}/{total}")
        logger.info(f"   Data saved to: {OUTPUT_FILE}")
        logger.info("=" * 60)
        
        if final_failed_count > total * 0.1:
            logger.warning(
                f"  High failure rate ({final_failed_count}/{total}). "
                f"Consider increasing REQUEST_DELAY (currently {REQUEST_DELAY}s) "
                f"or reducing max_concurrent (currently {max_concurrent})"
            )


# ============================================================================
# API endpoint discovery (optional): enumerate network calls via Playwright
# to transition from browser automation to direct HTTP requests where possible.
# ============================================================================

async def discover_api_endpoints(url: str):
    """
    Inspect network activity with Playwright to identify candidate API endpoints.

    Returns:
        A list of endpoint descriptors (URL, method, status, headers) discovered during navigation.
    """
    from playwright.async_api import async_playwright
    
    api_endpoints = []
    
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        
        async def handle_response(response):
            """Record responses whose URLs suggest API or GraphQL traffic."""
            if 'api' in response.url.lower() or 'graphql' in response.url.lower():
                api_endpoints.append({
                    'url': response.url,
                    'method': response.request.method,
                    'status': response.status,
                    'headers': dict(response.headers)
                })
                try:
                    body = await response.json()
                    logger.info(f"Found API endpoint: {response.url}")
                    logger.info(f"Response preview: {str(body)[:200]}")
                except:
                    pass
        
        page.on('response', handle_response)
        await page.goto(url, wait_until='networkidle')
        await asyncio.sleep(3)  # allow deferred requests to complete
        await browser.close()
    
    return api_endpoints


# ============================================================================
# Quick probe to verify whether static HTML extraction is sufficient for a URL.
# ============================================================================

async def test_if_bs4_works(test_url: str):
    """
    Probe a single profile and report whether HTML + GraphQL retrieval succeeds.

    Guidance:
    - If reviews are zero but metadata is present, reviews are likely client-rendered; use GraphQL path.
    - If both fail, fall back to endpoint discovery or headless automation.
    """
    async with httpx.AsyncClient() as client:
        result = await scrape_professor_bs4(client, test_url)
        
        if result:
            print(f"\n{'='*60}")
            print(f"Test Results for: {test_url}")
            print(f"{'='*60}")
            print(f"Name: {result['name']}")
            print(f"Department: {result['department']}")
            print(f"Rating: {result['rating']}")
            print(f"Reviews found: {len(result['reviews'])}")
            
            if len(result['reviews']) > 0:
                print(f"\n BeautifulSoup WORKS! Found {len(result['reviews'])} reviews")
                print("You can use the BeautifulSoup scraper.")
            else:
                print(f"\n  BeautifulSoup found NO reviews")
                print("Reviews are likely loaded via JavaScript.")
                print("Options:")
                print("  1. Use Playwright (test_playwright.py)")
                print("  2. Discover API endpoints (run discover_api_endpoints)")
                print("  3. Use hybrid approach (Playwright to find API, then BeautifulSoup)")
            
            print(f"{'='*60}\n")
        else:
            print(f" Failed to fetch page: {test_url}")


# Example usage (script entry point).
if __name__ == "__main__":
    # ------------------------------------------------------------
    # Step 1: sanity check for a single URL (static + GraphQL path)
    # ------------------------------------------------------------
    test_url = "https://www.ratemyprofessors.com/professor/1004624"
    
    print("=" * 60)
    print("Testing if BeautifulSoup can extract data...")
    print("=" * 60)
    await test_if_bs4_works(test_url)

    
    # ------------------------------------------------------------
    # Step 2: run the full pipeline on an explicit URL list
    # (paste your professor_list here or import it from elsewhere)
    # ------------------------------------------------------------
    professor_list 
    
    if professor_list:
        print("\n" + "=" * 60)
        print(f"Starting full scrape of {len(professor_list)} professors...")
        print(f"Rate limiting: {REQUEST_DELAY}s delay between requests")
        print(f"Max concurrent: 5 (adjustable - lower = safer from blocking)")
        print("=" * 60)
        # Begin conservatively; scale up if your environment tolerates it.
        await scrape_all_bs4(professor_list, max_concurrent=5)
    else:
        print("\n  Please add your professor_list to run the full scrape!")


2025-11-10 14:54:52,165 - INFO - Scraping: https://www.ratemyprofessors.com/professor/1004624


Testing if BeautifulSoup can extract data...


2025-11-10 14:54:52,554 - INFO - HTTP Request: GET https://www.ratemyprofessors.com/professor/1004624 "HTTP/1.1 200 OK"
2025-11-10 14:54:52,572 - INFO - Found 55 reviews using selector: [class*='Rating__']
2025-11-10 14:54:52,580 - INFO - Fetching all reviews via GraphQL for professor 1004624...
2025-11-10 14:54:52,661 - INFO - HTTP Request: POST https://www.ratemyprofessors.com/graphql "HTTP/1.1 200 OK"
2025-11-10 14:54:52,662 - INFO - Fetched 8 reviews via GraphQL for professor 1004624
2025-11-10 14:54:52,662 - INFO - Extracted 8 reviews from https://www.ratemyprofessors.com/professor/1004624 (via GraphQL)
2025-11-10 14:54:52,668 - INFO - Already scraped: 103
2025-11-10 14:54:52,669 - INFO - Remaining to scrape: 3666
2025-11-10 14:54:52,669 - INFO - Rate limiting: 1.0s delay, max 5 concurrent requests
2025-11-10 14:54:52,669 - INFO - Will handle 429 errors with exponential backoff (max 3 retries)
2025-11-10 14:54:52,686 - INFO - Scraping: https://www.ratemyprofessors.com/professor/30


Test Results for: https://www.ratemyprofessors.com/professor/1004624
Name: AmyLubitow
Department: Sociology
Rating: 4.8
Reviews found: 8

 BeautifulSoup WORKS! Found 8 reviews
You can use the BeautifulSoup scraper.


Starting full scrape of 3769 professors...
Rate limiting: 1.0s delay between requests
Max concurrent: 5 (adjustable - lower = safer from blocking)


2025-11-10 14:54:53,028 - INFO - HTTP Request: GET https://www.ratemyprofessors.com/professor/3007827 "HTTP/2 200 OK"
2025-11-10 14:54:53,046 - INFO - Found 55 reviews using selector: [class*='Rating__']
2025-11-10 14:54:53,056 - INFO - Fetching all reviews via GraphQL for professor 3007827...
2025-11-10 14:54:53,059 - INFO - HTTP Request: GET https://www.ratemyprofessors.com/professor/759386 "HTTP/2 200 OK"
2025-11-10 14:54:53,061 - INFO - HTTP Request: GET https://www.ratemyprofessors.com/professor/3040249 "HTTP/2 200 OK"
2025-11-10 14:54:53,063 - INFO - HTTP Request: GET https://www.ratemyprofessors.com/professor/1851743 "HTTP/2 200 OK"
2025-11-10 14:54:53,064 - INFO - HTTP Request: GET https://www.ratemyprofessors.com/professor/2111762 "HTTP/2 200 OK"
2025-11-10 14:54:53,079 - INFO - Found 55 reviews using selector: [class*='Rating__']
2025-11-10 14:54:53,087 - INFO - Fetching all reviews via GraphQL for professor 759386...
2025-11-10 14:54:53,103 - INFO - Found 55 reviews using se